# Preliminary support for predictive coding  

If inhibitory interneurons subtract a prediction of ganglion firing, we should expect higher correlations between the ganglion cells and connected inhibitory interneurons if we were able to silence inhibitory communication to the ganglion cells (but not alter the inhibitory membrane potential).  

We can essentially do this with intracellular record & playback experiments. This notebook will help provide some quantitative intuition for what direction correlations should change, as a function of stimulus statistics.

In [46]:
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import sem, pearsonr
from image_processing_functions import load_images
from lnl_model_functions import gaussian
from scipy.signal import convolve2d
import os

# This is a bit of magic to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
from matplotlib.pyplot import *

# note that nonposx(y) for log plots will no longer work with this package
import mpld3
#mpld3.enable_notebook()

from pylab import rcParams
rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate stimulus  

All of our receptive fields are 1d in space, so let the stimulus be space x time.

In [14]:
stimulus_type = 'white-noise'
num_frames = 1000

if stimulus_type == 'natural-scenes':
    im_dir = os.path.expanduser('~/Documents/Natural_Images/RawData/cd13A') # flood plain, water, horizon
    patch_size = None
    num_images = 100
    stimulus = load_images(im_dir, num_images, patch_size, square=True, normalize='divisive')

elif stimulus_type == 'white-noise':
    stimulus = np.random.randn(num_frames,32,32) 

# Create model  

Model should have multiple excitatory channels where a subset of the excitation flows into an inhibitory pathway. A subset of the excitation and inhibition should then combine to give a thresholded model prediction (ganglion response).  

To make this somewhat realistic, I'll use real bipolar and amacrine cell receptive fields, and ganglion cells will be a linear combination of them.

#### Get bipolar and amacrine receptive fields

In [40]:
def get_bipolar_rfs():
    # spatialDelta = 0.022 # mm
    data_path  = os.path.expanduser('~/Documents/Stanford/00 Baccus Lab/Surround size project/david_receptive_fields')
    file_name1 = data_path + '/B1.txt'
    file_name2 = data_path + '/B2.txt'
    data_b1    = np.loadtxt(file_name1, delimiter="\t") # 50 time x 100 space
    data_b2    = np.loadtxt(file_name2, delimiter="\t") # 50 time x 100 space
    data_b     = [data_b1.T, data_b2.T] # space x time
    return data_b

def get_amacrine_rfs():
    # spatialDelta = 0.027 # mm
    data_path = os.path.expanduser('~/Documents/Stanford/00 Baccus Lab/Surround size project/david_receptive_fields')
    file_name = data_path + '/H1A2.txt'
    data_ha   = np.loadtxt(file_name, delimiter="\t")
    data_ha   = data_ha.reshape((3,50,80))
    data_a    = data_ha[1:, :, :] # first cell in this file is a horizontal cell
    rfs       = []
    for a in data_a:
        rfs.append(a.T)
    return rfs
    

#### Construct model using bipolar and amacrine receptive fields

In [77]:
def ganglion_population_response(stimulus, num_ganglion_cells=10, threshold=1):
    '''
    Stimulus should be space x time.
    Threshold should be in standard deviations.
    Returns a response ganglion population x time
    '''
    if len(stimulus.shape) != 2:
        raise Exception('Stimulus needs to be 2 dimensional.')
    
    # loading receptive fields as (space in degrees, sensitivity) tuples
    bipolar_rfs = get_bipolar_rfs()
    amacrine_rfs = get_amacrine_rfs()
    
    # get the linear response of bipolar and amacrine cells
    bipolar_response_hd = [convolve2d(stimulus, bipolar, mode='same') for bipolar in bipolar_rfs]
    amacrine_response_hd = [convolve2d(stimulus, amacrine, mode='same') for amacrine in amacrine_rfs]
    
    # subsample in space
    bipolar_responses = [response[::10,:] for response in bipolar_response_hd]
    amacrine_responses = [response[::10,:] for response in amacrine_response_hd]
    
    # for now just pick one type of bipolar cell and one type of amacrine cell
    bipolar_response = bipolar_responses[0]
    amacrine_response = amacrine_responses[0]
    
    # how many bipolar cells and how many amacrine cells are connected to the ganglion cells,
    # and with what strength?
    bip_const = 1.
    ama_const = 78./22.
    bip_projection = np.tile([np.exp(-abs(s)/bip_const) for s in np.linspace(-22,22,bipolar_response.shape[1])], (stimulus.shape[1],1)).T
    ama_projection = np.tile([np.exp(-abs(s)/ama_const) for s in np.linspace(-22,22,amacrine_response.shape[1])], (stimulus.shape[1],1)).T
    
    # compute ganglion response
    excitation = convolve2d(bipolar_response, bip_projection, mode='same')
    inhibition = convolve2d(amacrine_response, ama_projection, mode='same')
    ganglion_response = excitation - inhibition
    
    # threshold response
    return np.where(ganglion_response > threshold * np.std(ganglion_response), ganglion_response, 0)

In [78]:
ganglions = ganglion_population_response(stimulus)

Exception: Stimulus needs to be 2 dimensional.